In [ ]:
import pandas as pd
import numpy as np
import json
import os

# ==========================================
# Configuration
# ==========================================
JSON_PATH = '../country_list.json'
CSV_PATH = '../../data_origin/Country_relation_United Nations General Assembly Ideal Points/dataverse_files/AgreementScores.csv'
OUTPUT_DIR = '../../data'
YEAR_START, YEAR_END = 2005, 2023
DIST_COL = 'IdealPointDistance'   

os.makedirs(OUTPUT_DIR, exist_ok=True)

def check_data_holes(df: pd.DataFrame, year_range: int) -> None:
    pair_counts = df.groupby(['ccode1', 'ccode2'])['year'].count()
    total_pairs = len(pair_counts)
    incomplete_pairs = pair_counts[pair_counts < year_range]
    
    print(f"=== Data Hole Report ({YEAR_START}-{YEAR_END}) ===")
    print(f"Total interacting pairs: {total_pairs}")
    print(f"Pairs with full coverage ({year_range} years): {total_pairs - len(incomplete_pairs)}")
    print(f"Pairs with missing years (Holes): {len(incomplete_pairs)}")

def build_average_matrix(df: pd.DataFrame, target_map: dict) -> pd.DataFrame:
    """
    Computes average distance matrix, performs internal normalization, 
    and ensures 49x49 symmetry.
    """
    target_ccodes = list(target_map.keys())
    
    # 1. Symmetrize data
    # [注意] 必须保留 'year' 用于后续分组
    df_sym = pd.concat([
        df[['ccode1', 'ccode2', 'year', DIST_COL]],
        df.rename(columns={'ccode1': 'ccode2', 'ccode2': 'ccode1'})[['ccode1', 'ccode2', 'year', DIST_COL]]
    ])
    
    # ================================================================
    # [优化步骤] 针对每一个年份都进行归一化 (Yearly Normalization)
    # 使用 transform 替代 apply，消除 Warning 并大幅提升速度
    # ================================================================
    # 1. 计算每一年的最大值和最小值 (广播回原数据维度)
    grouped = df_sym.groupby('year')[DIST_COL]
    min_vals = grouped.transform('min')
    max_vals = grouped.transform('max')
    
    # 2. 计算分母 (Max - Min)
    denominator = max_vals - min_vals
    
    # 3. 执行反向归一化: (Max - x) / (Max - Min)
    # 使用 np.where 处理分母为 0 的极端情况 (即该年所有国家距离都相等)
    df_sym['norm_score'] = np.where(
        denominator == 0, 
        1.0,  # 如果 max == min，视为完全一致
        (max_vals - df_sym[DIST_COL]) / denominator
    )
    # ================================================================
    
    # 2. Calculate mean similarity (Average over years)
    # 计算 'norm_score' 的平均值
    df_avg = df_sym.groupby(['ccode1', 'ccode2'])['norm_score'].mean().reset_index()
    
    # 3. Pivot to matrix
    matrix = df_avg.pivot(index='ccode1', columns='ccode2', values='norm_score')
    matrix = matrix.reindex(index=target_ccodes, columns=target_ccodes)
    
    # 4. Handle Diagonal
    # 重新强制对角线为 0 (适合回归分析/排除自环)
    np.fill_diagonal(matrix.values, 0)
    
    # 5. Map to ISO codes and sort
    matrix.index = matrix.index.map(target_map)
    matrix.columns = matrix.columns.map(target_map)
    matrix.sort_index(axis=0, inplace=True)
    matrix.sort_index(axis=1, inplace=True)
    
    return matrix

# ==========================================
# Main Execution
# ==========================================

with open(JSON_PATH, 'r', encoding='utf-8') as f:
    target_map = {item['cow']: item['iso'] for item in json.load(f)['countries']}

print(f"读取数据列: ccode1, ccode2, year, {DIST_COL} ...")
df = pd.read_csv(CSV_PATH, usecols=['ccode1', 'ccode2', 'year', DIST_COL])

df = df[
    (df['year'] >= YEAR_START) & 
    (df['year'] <= YEAR_END) & 
    (df['ccode1'].isin(target_map.keys())) & 
    (df['ccode2'].isin(target_map.keys()))
]

check_data_holes(df, year_range=(YEAR_END - YEAR_START + 1))

avg_matrix = build_average_matrix(df, target_map)

# 修改了文件名以体现逻辑变化
save_path = f"{OUTPUT_DIR}/1-1-norm_ideal_sim_{YEAR_START}_{YEAR_END}.csv"
avg_matrix.to_csv(save_path)
print(f"\n已保存反转归一化后的矩阵至: {save_path}")
print(avg_matrix.iloc[:5, :5])

读取数据列: ccode1, ccode2, year, IdealPointDistance ...
=== Data Hole Report (2005-2023) ===
Total interacting pairs: 2352
Pairs with full coverage (19 years): 2352
Pairs with missing years (Holes): 0

已保存反转归一化后的矩阵至: ../../data/1-1-norm_ideal_sim_2005_2023.csv
ccode2       ARG       AUS       AUT       BEL       BGR
ccode1                                                  
ARG     0.000000  0.575784  0.740126  0.664844  0.668624
AUS     0.575784  0.000000  0.835766  0.911048  0.907268
AUT     0.740126  0.835766  0.000000  0.924826  0.928606
BEL     0.664844  0.911048  0.924826  0.000000  0.982361
BGR     0.668624  0.907268  0.928606  0.982361  0.000000
